<a href="https://colab.research.google.com/github/Wardmisp/Bird-Eye-View/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
from torchsummary import summary

Solve the padding = 'same' issue

In [3]:
def padding_same(output_dim,input_dim, kernel, stride):
  return (int((output_dim-((input_dim-kernel)/stride+1))*stride/2))

In [8]:
class Net(torch.nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        # 1
        self.cnn_layers1 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 64, kernel_size=8, stride=2, padding=[padding_same(48,96,8,2),padding_same(72,144,8,2)]),
            torch.nn.LeakyReLU(negative_slope=0.2, inplace=True))
        
        # 2
        self.cnn_layers2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=6, stride=2, padding=[padding_same(24,48,6,2),padding_same(36,72,6,2)]),
            torch.nn.BatchNorm2d(128),
            torch.nn.LeakyReLU(negative_slope=0.2, inplace=True))
        
        # 3
        self.cnn_layers3 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=[padding_same(12,24,4,2),padding_same(18,36,4,2)]),
            torch.nn.BatchNorm2d(256),
            torch.nn.LeakyReLU(negative_slope=0.2, inplace=True),
            torch.nn.Upsample(scale_factor=2),
            torch.nn.Conv2d(256, 128, kernel_size=6, stride=2, padding=[20,14]),
            torch.nn.BatchNorm2d(128))
        
        # 4
        self.cnn_layers4 = torch.nn.Sequential(
            torch.nn.LeakyReLU(negative_slope=0.2, inplace=True),
            torch.nn.Upsample(scale_factor=2),
            torch.nn.Conv2d(256, 64, kernel_size=6, stride=2, padding=1),
            torch.nn.BatchNorm2d(64))
        
        #5
        self.cnn_layers5 = torch.nn.Sequential(
            torch.nn.LeakyReLU(negative_slope=0.2, inplace=True),
            torch.nn.Upsample(scale_factor=2),
            torch.nn.Conv2d(128, 7, kernel_size=8, stride=2, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.Softmax())

    # Defining the forward pass concat here
    def forward(self, x):
        x = self.cnn_layers1(x)
        y = self.cnn_layers2(x)
        print(y.shape)
        z = self.cnn_layers3(y)
        print(z.shape)
        y = torch.cat((y,z),3)
        print(y.shape)
        y = self.cnn_layers4(y)
        x = torch.cat((x,y),0)
        x = self.cnn_layers5(x)
        return x

# defining the model
model = Net()    
print(model)
summary(model,(4, 144, 96))

Net(
  (cnn_layers1): Sequential(
    (0): Conv2d(4, 64, kernel_size=(8, 8), stride=(2, 2), padding=[3, 3])
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (cnn_layers2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(6, 6), stride=(2, 2), padding=[2, 2])
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (cnn_layers3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=[1, 1])
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Upsample(scale_factor=2.0, mode=nearest)
    (4): Conv2d(256, 128, kernel_size=(6, 6), stride=(2, 2), padding=[20, 14])
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cnn_layers4): Sequential(
    (0): LeakyReLU(negative_slope=0.2, inplace=True)
    (1): Upsample(scale_facto

RuntimeError: ignored